In [12]:
from calo_cluster.evaluation.experiments.base_experiment import BaseExperiment
from calo_cluster.evaluation.experiments.base_offset_experiment import BaseOffsetExperiment
from pathlib import Path 
from calo_cluster.clustering.meanshift import MeanShift
from tqdm.auto import tqdm
from calo_cluster.evaluation.metrics.classification import mIoU
from calo_cluster.evaluation.metrics.instance import PanopticQuality
import numpy as np
import plotly
import plotly.express as px
import pandas as pd
from calo_cluster.evaluation.utils import get_palette

In [8]:
exp = BaseOffsetExperiment('1zs7gjy9')
evts = exp.get_events('val', n=100)

run_dir = /global/cscratch1/sd/schuya/calo_cluster/training_outputs/wandb/run-20211103_115852-1zs7gjy9
no checkpoint name given, using /global/cscratch1/sd/schuya/calo_cluster/training_outputs/hgcal-spvcnn/1zs7gjy9/checkpoints/last.ckpt.


Global seed set to 1588147245


  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
from tqdm.auto import tqdm
bws = [1e-3, 1e-2, 1e-1]
rets = {}
for bw in bws:
    print(f'bw = {bw}')
    clusterer = MeanShift(use_gpu=False, bandwidth=bw, use_semantic=True, ignore_semantic_labels=(0,))
    pq = PanopticQuality(num_classes=2, ignore_index=None, ignore_semantic_labels=(0,))
    for evt in tqdm(evts):
        evt.pred_instance_labels = clusterer.cluster(embedding=evt.embedding, semantic_labels=evt.pred_semantic_labels)
        pq.add((evt.pred_semantic_labels, evt.pred_instance_labels), (evt.input_event['hit'].to_numpy(), evt.input_event['trackId'].astype(int).to_numpy()))
    rets[bw] = pq.compute()
    print(f'(bw = {bw}) pq = {rets[bw]}')

bw = 0.001


  0%|          | 0/100 [00:00<?, ?it/s]

(bw = 0.001) pq = {'sq': array([-1.        ,  0.98438392]), 'rq': array([-1.        ,  0.54214518]), 'pq': 0.5336789970592788, 'tq': array([-1.        ,  0.55773518]), 'wrq': array([-1.        ,  0.62098158]), 'wpq': 0.6112842838971831, 'wtq': array([-1.      ,  0.621337])}
bw = 0.01


  0%|          | 0/100 [00:00<?, ?it/s]

(bw = 0.01) pq = {'sq': array([-1.        ,  0.98223458]), 'rq': array([-1.       ,  0.5487778]), 'pq': 0.5390285336772895, 'tq': array([-1.        ,  0.55550602]), 'wrq': array([-1.        ,  0.62715194]), 'wpq': 0.6160103172770676, 'wtq': array([-1.        ,  0.62693463])}
bw = 0.1


  0%|          | 0/100 [00:00<?, ?it/s]

(bw = 0.1) pq = {'sq': array([-1.        ,  0.96872281]), 'rq': array([-1.        ,  0.56183206]), 'pq': 0.5442595346412138, 'tq': array([-1.        ,  0.49219795]), 'wrq': array([-1.        ,  0.64542212]), 'wpq': 0.6252351283058771, 'wtq': array([-1.        ,  0.64009434])}


In [30]:
n = 6
evt = evts[n]
d = {'eta': evt.input_event['eta'], 'phi': evt.input_event['phi'], 'energy': evt.input_event['energy'], 'hit': evt.input_event['hit'], 'id': evt.input_event['PFcluster0Id'], 'eta_pred': evt.embedding[:,0], 'phi_pred': evt.embedding[:,1]}
plot_df = pd.DataFrame(d)
plot_df['id'] = plot_df['id'].astype(str)
plot_df = plot_df[plot_df['hit'] == 1]
px.scatter(plot_df, x='eta', y='phi', size='energy', color='id', color_discrete_sequence=get_palette(plot_df['id']))

In [33]:
n = 4
evt = evts[n]
d = {'eta': evt.input_event['eta'], 'phi': evt.input_event['phi'], 'energy': evt.input_event['energy'], 'hit': evt.input_event['hit'], 'id': evt.input_event['trackId'], 'eta_pred': evt.embedding[:,0], 'phi_pred': evt.embedding[:,1]}
plot_df = pd.DataFrame(d)
plot_df['id'] = plot_df['id'].astype(str)
px.scatter(plot_df, x='eta', y='phi', size='energy', color='id', color_discrete_sequence=get_palette(plot_df['id']))

In [34]:
px.scatter(plot_df, x='eta_pred', y='phi_pred', size='energy', color='id', color_discrete_sequence=get_palette(plot_df['id']))

In [26]:
n = 4
evt = evts[n]
d = {'eta': evt.input_event['eta'], 'phi': evt.input_event['phi'], 'energy': evt.input_event['energy'], 'hit': evt.input_event['hit'], 'id': evt.pred_instance_labels, 'eta_pred': evt.embedding[:,0], 'phi_pred': evt.embedding[:,1]}
plot_df = pd.DataFrame(d)
plot_df['id'] = plot_df['id'].astype(str)
px.scatter(plot_df, x='eta_pred', y='phi_pred', size='energy', color='id', color_discrete_sequence=get_palette(plot_df['id']))

In [3]:
events = exp.get_events('test', n=1000)

  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
# iou for pf:
for evt in evts:
    evt.pred_semantic_labels = evt.input_event['pf_hit']

iou = mIoU(evts, num_classes=2, ignore_index=0, semantic_label='hit', reduction='none')
print(iou.mean())

  0%|          | 0/100 [00:00<?, ?it/s]


mIoU = tensor([0.7399])
tensor(0.7399)


In [5]:
clusterer = MeanShift(bandwidth=0.1, use_semantic=True, ignore_semantic_labels=(0,), use_gpu=False)
for evt in tqdm(events):
    semantic_truth = evt.input_event['hit'].to_numpy()
    evt.pred_instance_labels = clusterer.cluster(embedding=evt.embedding, semantic_labels=semantic_truth)
outputs = [(evt.input_event['hit'].to_numpy(), evt.pred_instance_labels) for evt in events]
targets = [(evt.input_event['hit'].to_numpy(), evt.input_event['trackId'].to_numpy()) for evt in events]
weights = [evt.input_event['genE'].to_numpy() for evt in events]

In [ ]:
from calo_cluster.evaluation.studies.functional import response
cluster_response, event_response = response(outputs, targets, weights, match_highest=False, num_classes=2, ignore_semantic_labels=(0,))

In [ ]:
unmatched_mask = cluster_response[1]['energy response'] == 0.0
unmatched_energy = cluster_response[1].loc[unmatched_mask, 'energy']
matched_energy = cluster_response[1].loc[~unmatched_mask, 'energy']
unmatched_hist, bin_edges = np.histogram(unmatched_energy, bins=20, range=(0, 10))
matched_hist, _ = np.histogram(matched_energy, bins=20, range=(0, 10))
efficiency = matched_hist / (matched_hist + unmatched_hist)
x = [(bin_edges[i] + bin_edges[i+1]) / 2 for i in range(len(bin_edges) - 1)]

In [ ]:
px.scatter(y=efficiency, x=x)